In [1]:
#!/usr/bin/env python3

from http.client  import HTTPConnection
from urllib.parse import urlencode
from arcgis.mapping import create_symbol
import pandas as pd
import numpy as np
import json
from datetime import datetime, date, time

#------------------------------------------------------
# Runs SPARQL query at SPARQL endpoint and
# return results as a Python 'dict' (in the SPARQL1.1 results format)
# (for SPARQL1.1 results format refer: https://www.w3.org/TR/sparql11-results-json)
#
#       sparql_endpoint: 'host:port' ex: '192.168.0.64:7070', 'data.nobelprize.org'
#       sparql_query: ex: 'select (count(*) as ?c) {?s?p?o}'
#       fmt - optional argument, if specified, returns results in a raw string format
#              possiblea values ('csv','json','xml'), any other format will be treated as 'json'
#------------------------------------------------------
#
def run_query(sparql_endpoint,sparql_query,fmt=None):
   # create HTTP connection to SPARQL endpoint
   conn = HTTPConnection(sparql_endpoint,timeout=100) #may throw HTTPConnection exception
   # urlencode query for sending
   docbody = urlencode({'query':sparql_query})
   # request result in json
   hdrs = {'Accept': 'application/sparql-results+json',
           'Content-type': 'application/x-www-form-urlencoded'}
   raw = False
   if fmt is not None:
      raw = True
      if fmt in ('xml','XML'):
         hdrs['Accept'] = 'application/sparql-results+xml'
      elif fmt in ('csv','CSV'):
         hdrs['Accept'] = 'text/csv, application/sparql-results+csv'

   # send post request
   conn.request('POST','/sparql',docbody,hdrs) #may throw exception

   # read response
   resp = conn.getresponse()
   if 200 != resp.status:
      errmsg = resp.read()
      conn.close()
      raise Exception('Query Error',errmsg)  # query processing errors - syntax errors, etc.

   # content-type header, and actual response data
   ctype = resp.getheader('content-type','text/html').lower()
   result = resp.read().lstrip()
   conn.close()

   # check response content-type header
   if raw or ctype.find('json') < 0:
      return result      # not a SELECT?

   # convert result in JSON string into python dict
   return json.loads(result)


#------------------------------------------------------
# Returns pandas DataFrame from the results of running a sparql_query at sparql_endpoint
#       sparql_endpoint: 'host:port' ex: '192.168.0.64:7070', 'data.nobelprize.org'
#       sparql_query: ex: 'select (count(*) as ?c) {?s?p?o}'
#------------------------------------------------------
#
def create_dataframe(sparql_endpoint,sparql_query):
   # run query
   result = run_query(sparql_endpoint,sparql_query)  # may throw exception
   # result is in SPARQL results format refer: https://www.w3.org/TR/sparql11-results-json/
   cols = result.get('head',{}).get('vars',[])
   rows = result.get('results',{}).get('bindings',[])

   # extract types and columnar data for rows
   coltype = {}
   nptype = {}
   coldata = {}
   for col in cols:
      coltype[col] = None
      coldata[col] = []
      nptype[col] = None

   # for all rows, save (columnar) data in coldata[] for each col
   for row in rows:
      for col in cols:
         cell = row.get(col,None)
         if cell is None:  # unbound value
            val = None
            if coltype[col] in ('byte','short','int','integer','float','double','decimal'):
               val = np.nan #missing numeric values as NaN
            coldata[col].append(val)
            continue
         # compute type and datum
         pdval = cell.get('value','')
         vtype = cell.get('type','')
         langtag = cell.get('xml:lang','')
         typeuri = cell.get('datatype','')
         pdtype = 'object'
         if vtype == 'uri':
            pdval = '<'+pdval+'>'
         elif langtag != '':
            pdval = '"'+pdval+'"@'+langtag
            coltype[col] = 'object'
         elif typeuri != '':
            #vtype in ('typed-literal')
            typeuri = typeuri.replace('http://www.w3.org/2001/XMLSchema#','')
            coltype[col] = typeuri if (coltype[col] is None or coltype[col] == typeuri) else 'object'
            pdtype,pdval = typed_value(typeuri,pdval)
         nptype[col] = pdtype if (coltype[col] != 'object') else 'object'
         coldata[col].append(pdval) # columnar data
   # instantiate DataFrame
   npdata = {}
   for col in cols:
      npdata[col] = np.array(coldata[col],dtype=np.dtype(nptype[col]))
   return pd.DataFrame(columns=cols,data=npdata)

# util: convert literal val into typed-value based on the typeuri
def typed_value(typeuri,val):
   # {"duration", ColTypeDuration},
   if typeuri in ('boolean'):
      return np.bool, 'true' == val
   elif typeuri in ('byte'):
      return np.byte, np.int8(val)
   elif typeuri in ('short'):
      return np.short, np.short(val)
   elif typeuri in ('integer','int','nonNegativeInteger'):
      return np.intc, np.int(val)
   elif typeuri in ('long'):
      return np.int_, np.int_(val)
   elif typeuri in ('float'):
      return np.single, np.float32(val)
   elif typeuri in ('double', 'decimal'):
      return np.double, np.float64(val)
   elif typeuri in ('dateTime'):
      return np.datetime64, datetime.fromisoformat(val)
   elif typeuri in ('date'):
      return pd.date, date.fromisoformat(val)
   elif typeuri in ('time'):
      return pd.time, time.fromisoformat(val)
   return 'object', val


In [2]:
constructQuery = '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix : <https://ontologies.semanticarts.com/raw_data#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix skos:    <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { 
  	?orig fl:hasRouteTo ?dest .
  	?orig rdfs:label ?origCode . 
  	?dest rdfs:label ?destCode . 
  	fl:hasRouteTo rdfs:label "hasRouteTo" . 
}  
WHERE { GRAPH <airline_flight_network> {{
SELECT 
?orig ?dest ?origCode ?destCode ?miles 
    WHERE { 
        ?orig fl:terminalCode ?origCode .
        ?orig fl:hasRouteTo ?dest .
        ?dest fl:terminalCode ?destCode .
        << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
        FILTER (?miles < 400)
        FILTER (?origCode = 'BOS')
  }
      }}}
'''

In [3]:
# def createAirportQuery(origin):
#     return '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# prefix : <https://ontologies.semanticarts.com/raw_data#>
# prefix fl: <https://ontologies.semanticarts.com/flights/>
# prefix owl: <http://www.w3.org/2002/07/owl#>
# prefix skos:    <http://www.w3.org/2004/02/skos/core#>
# CONSTRUCT { 
#   	?orig fl:hasRouteTo ?dest .
#   	?orig rdfs:label ?origCode . 
#   	?dest rdfs:label ?destCode . 
#   	fl:hasRouteTo rdfs:label "hasRouteTo" . 
# }  
# WHERE { GRAPH <airline_flight_network> {{
# SELECT 
# ?orig ?dest ?origCode ?destCode ?miles 
#     WHERE { 
#         ?orig fl:terminalCode ?origCode .
#         ?orig fl:hasRouteTo ?dest .
#         ?dest fl:terminalCode ?destCode .
#         << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
#         FILTER (?miles < 400)
#         FILTER (?origCode = 'BOS')
#   }
#       }}}
# '''

labelDict = {}
def createGraph(query):
#     print('Inside')
#     query = createAirportQuery(origin)

# change variable names:

    data = run_query('localhost:7070',query)

    decodedData = data.decode("utf-8")

    a = decodedData.split(".\n")
    a = a[1: len(a)-1]
    triples = set()
    for ele in a:
        ele = ele[:len(ele)-1]
        triples.add(ele)
    # nodeNames = set()
    
    graph = []
    nodes = set()
    edges = set()

    # for ele in triples:
    #     arr = ele.split(' ')
    #     print(arr[0])

    for ele in triples:
        arr = ele.split(' ')
        p = arr[1].split('/')
        p = p[len(p)-1]
        p = p[:len(p)-1]
    
        if "label" in p:
            s = arr[0]
            o = arr[2][1:len(arr[2])-1]
            labelDict[s.lower()] = o
    # print(labelDict)

    for ele in triples:
        arr = ele.split(' ') 
        s = arr[0]
        p = arr[1]
        o = arr[2]
    
        nodes.add(s)
    
        if arr[2][0] == '<' and arr[2][len(arr[2])-1] == '>':
            nodes.add(o)
            graph.append({'S':s,'P':p,'O':o})
            edges.add(p)
        
    nodes = nodes - edges
    
    elements = []
    for node in nodes:
        elements.append({'data': {'id': node,
                                  'label': labelDict.get(node.lower(),node),
                                  'expanded':False
                                 }
                       })
    for ele in graph:    
        elements.append({'data':{'source': ele['S'],
                                 'target': ele['O'],
                                 'label': ele['P'],
                                 'labelLabel': labelDict.get(ele['P'].lower(),ele['P']),
                                 'sourceLabel': labelDict.get(ele['S'].lower(),ele['S']),
                                 'targetLabel': labelDict.get(ele['O'].lower(),ele['O'])
                               }
                       })
    return elements

def getEdgeLabel(origin):
    return 'From Airport: ' + origin

def getNeighbours(origin):
    query = '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?s ?p ?obj ?obj_type ?obj_label from <airline_flight_network>
        where {
            ?s ?p ?obj .
			optional {  ?obj a ?obj_type . }
    		optional {	?obj rdfs:label ?obj_label .  }
      VALUES ?s { 	''' + origin + '''} 
  filter(ISIRI(?obj_type))   
  }
#	order by ?s 
    limit 10'''
    df = create_dataframe('localhost:7070',query)
    return df

# def generateNodes(df):
  
import math    
def generateNodes(df):
    
    # verify to check whether new nodes can be repetitve and accordingly create a set of nodes and then a list of newNodes
    newNodes = []
    
    # insert label into dict later
    
    for i in df.index:
    
        
    
        
        
        newNodes.append({'data': {'id': df['obj'][i],
                                  'label': labelDict.get(df['obj'][i].lower(),df['obj'][i]),
                                  'expanded': False
                                  }
                        }) 
#     print("Newnodes",newNodes)        
    return newNodes 

def generateEdges(df):
    newEdges = []
    
    # insert label into dict later
    
    for i in df.index:
        
        newEdges.append({'data': {'source': df['s'][i],
                                  'target': df['obj'][i],
                                  'label': df['p'][i],
                                  'labelLabel': labelDict.get(df['p'][i].lower(),df['p'][i]),
                                  'sourceLabel': labelDict.get(df['s'][i].lower(),df['s'][i]),
                                  'targetLabel': labelDict.get(df['obj'][i].lower(),df['obj'][i]) 
            
        }
            
        })
        
    return newEdges    
        

In [4]:
elements = createGraph(constructQuery)
df = getNeighbours('<https://data.semanticarts.com/flights/_Airport_BOS>')
# x = generateNodes(df)
# x
df
# for i in df.index:
#         print(df['s'][i]," ",df['p'][i]," ",df['obj'][i])
        
# generateNodes(df)


,s,p,obj,obj_type,obj_label
0,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
1,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
2,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
3,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
4,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
5,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
6,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
7,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
8,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN
9,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/h...,<https://data.semanticarts.com/flights/_Airpor...,<https://ontologies.semanticarts.com/flights/A...,NaN


In [5]:
# import math
# def generateNodes(df):
#     for i in df.index:
    
#         t = df['obj_type'][i]

#         # insert label into dict
    
#         newNodes = []
#         if "Airport" in t:
#             newNodes.append({'data': {'id': df['obj'][i],
#                                       'label': labelDict.get(df['obj'][i].lower(),df['obj'][i])
#                                      }
#                            }) 
#     print(newNodes)        
#     return newNodes    
    
   
# x = generateNodes('BOS')
# print(x)

# print(math.isnan(df['obj_label'][5]))

In [6]:
import json
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

app = JupyterDash(__name__)

app.layout = html.Div([
     
    html.Div(className='a',children = [
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='circle',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
        ]
    )]),
#     html.Div(className='c',children = [
#     dcc.RadioItems(
#         id='dropdown-show-neighbors',
#         options=[
#             {'label': 'All neighbors', 'value': 'all'},
#             {'label': 'Neighbours with hasRouteTo', 'value': 'hRT'},
#             {'label': 'Neighbours with locatedIn', 'value': 'lI'}
#         ],
#         value='all',
#         labelStyle={'display': 'inline-block'}
#         ),
#      html.Button('Submit', id='button'),   
#     ]),
    html.Div(className='b',children = [
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'circle'},
        style={'width': '100%', 'height': '600px'},
        elements=elements,
        stylesheet = [
            {
            'selector': 'node',
            'style': {
                'content': 'data(label)',
                'background-color': '#58FAF4',
                }
            },
            {
            'selector': 'edge',
            'style': {
                'content': 'data(labelLabel)',
                'curve-style': 'bezier',
                'target-arrow-color': 'black',
                'target-arrow-shape': 'triangle',
                'line-color': 'black'
                }
            }
        ]
    )]),
    html.P(id='cytoscape-tapNodeData-output'),
    html.P(id='cytoscape-tapEdgeData-output')
])


@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapNodeData')])
def displayTapNodeData(data):
#     print('HH')
    if data:
        return "Airport selected: " + data['id']


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapEdgeData')]
             )
def displayTapEdgeData(data):
    if data:
        return "Connection between " + data['source'].upper() + " and " + data['target'].upper() + " with edge value: " + data['label']

    
@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

@app.callback(Output('cytoscape-update-layout', 'elements'),
              [Input('cytoscape-update-layout', 'tapNodeData')],
              [State('cytoscape-update-layout', 'elements')]
             )
def generate_elements(data,e):
#     print("ABCD",data)
#     print(type(e))
    if not data:
        return e
    
    if data['expanded'] == True:
        return e
    
#     print("Label",data)
    
    if data and e:
        
        #changing extended to True for the node
        for element in e:
            if data['id'] == element.get('data').get('id'):
                element['data']['expanded'] = True
#                 print("Curr Node", element)
                break
                
        nodeURI = data['id']
#         print('node',nodeURI)
#         print("Label",label)
#         print('Here', type(e))
        df = getNeighbours(nodeURI)
        nodes = generateNodes(df)
        edges = generateEdges(df)
#         node = {'data':{'id': '<https://data.semanticarts.com/flights/_Airport_KOL>', 'label': 'KOL'}}
#         edge = {'data': {'source': '<https://data.semanticarts.com/flights/_Airport_JFK>',
#                          'target': '<https://data.semanticarts.com/flights/_Airport_KOL>',
#                          'label': '<https://ontologies.semanticarts.com/flights/hasRouteTo>',
#                          'labelLabel': getEdgeLabel(data['label']),
#                          'sourceLabel': 'BOS',
#                          'targetLabel': 'KOL'}}
#         if node in e:
#             print("A")
#             return e
#         else:
            
#             print("B")
#             e.append(nodes)
#             e.append(edge)
#             return e

        for node in nodes:
            if node not in e:
                e.append(node)
                
        for edge in edges:
            if edge not in e:
                e.append(edge)        
                
    return e
                
            
            
port = '8099'
host = '127.0.0.1'

app.run_server(mode='inline',port = port, host = host)

# print('App running on http://'+ host +':' + port +'/')

In [18]:
import json
import os

from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

import dash_cytoscape as cyto
from demos import dash_reusable_components as drc

# Load extra layouts
cyto.load_extra_layouts()


# asset_path = os.path.join(
#     os.path.dirname(os.path.abspath(__file__)),
#     '..', 'assets'
# )

app = JupyterDash(__name__)
server = app.server


# ###################### DATA PREPROCESSING ######################
# Load data
with open('demos/data/sample_network.txt', 'r') as f:
    network_data = f.read().split('\n')

# We select the first 750 edges and associated nodes for an easier visualization
edges = network_data[:750]
nodes = set()

following_node_di = {}  # user id -> list of users they are following
following_edges_di = {}  # user id -> list of cy edges starting from user id

followers_node_di = {}  # user id -> list of followers (cy_node format)
followers_edges_di = {}  # user id -> list of cy edges ending at user id

cy_edges = []
cy_nodes = []

for edge in edges:
    if " " not in edge:
        continue

    source, target = edge.split(" ")

    cy_edge = {'data': {'id': source+target, 'source': source, 'target': target}}
    cy_target = {"data": {"id": target, "label": "User #" + str(target[-5:])}}
    cy_source = {"data": {"id": source, "label": "User #" + str(source[-5:])}}

    if source not in nodes:
        nodes.add(source)
        cy_nodes.append(cy_source)
    if target not in nodes:
        nodes.add(target)
        cy_nodes.append(cy_target)

    # Process dictionary of following
    if not following_node_di.get(source):
        following_node_di[source] = []
    if not following_edges_di.get(source):
        following_edges_di[source] = []

    following_node_di[source].append(cy_target)
    following_edges_di[source].append(cy_edge)

    # Process dictionary of followers
    if not followers_node_di.get(target):
        followers_node_di[target] = []
    if not followers_edges_di.get(target):
        followers_edges_di[target] = []

    followers_node_di[target].append(cy_source)
    followers_edges_di[target].append(cy_edge)

genesis_node = cy_nodes[0]
genesis_node['classes'] = "genesis"
default_elements = [genesis_node]

default_stylesheet = [
    {
        "selector": 'node',
        'style': {
            "opacity": 0.65,
            'z-index': 9999
        }
    },
    {
        "selector": 'edge',
        'style': {
            "curve-style": "bezier",
            "opacity": 0.45,
            'z-index': 5000
        }
    },
    {
        'selector': '.followerNode',
        'style': {
            'background-color': '#0074D9'
        }
    },
    {
        'selector': '.followerEdge',
        "style": {
            "mid-target-arrow-color": "blue",
            "mid-target-arrow-shape": "vee",
            "line-color": "#0074D9"
        }
    },
    {
        'selector': '.followingNode',
        'style': {
            'background-color': '#FF4136'
        }
    },
    {
        'selector': '.followingEdge',
        "style": {
            "mid-target-arrow-color": "red",
            "mid-target-arrow-shape": "vee",
            "line-color": "#FF4136",
        }
    },
    {
        "selector": '.genesis',
        "style": {
            'background-color': '#B10DC9',
            "border-width": 2,
            "border-color": "purple",
            "border-opacity": 1,
            "opacity": 1,

            "label": "data(label)",
            "color": "#B10DC9",
            "text-opacity": 1,
            "font-size": 12,
            'z-index': 9999
        }
    },
    {
        'selector': ':selected',
        "style": {
            "border-width": 2,
            "border-color": "black",
            "border-opacity": 1,
            "opacity": 1,
            "label": "data(label)",
            "color": "black",
            "font-size": 12,
            'z-index': 9999
        }
    }
]

# ################################# APP LAYOUT ################################
styles = {
    'json-output': {
        'overflow-y': 'scroll',
        'height': 'calc(50% - 25px)',
        'border': 'thin lightgrey solid'
    },
    'tab': {'height': 'calc(98vh - 80px)'}
}

app.layout = html.Div([
    

    html.Div(className='four columns', children=[
        dcc.Tabs(id='tabs', children=[
            dcc.Tab(label='Control Panel', children=[
                drc.NamedDropdown(
                    name='Layout',
                    id='dropdown-layout',
                    options=drc.DropdownOptionsList(
                        'random',
                        'grid',
                        'circle',
                        'concentric',
                        'breadthfirst',
                        'cose',
                        'cose-bilkent',
                        'dagre',
                        'cola',
                        'klay',
                        'spread',
                        'euler'
                    ),
                    value='grid',
                    clearable=False
                ),
                drc.NamedRadioItems(
                    name='Expand',
                    id='radio-expand',
                    options=drc.DropdownOptionsList(
                        'followers',
                        'following'
                    ),
                    value='followers'
                )
            ]),

            dcc.Tab(label='JSON', children=[
                html.Div(style=styles['tab'], children=[
                    html.P('Node Object JSON:'),
                    html.Pre(
                        id='tap-node-json-output',
                        style=styles['json-output']
                    ),
                    html.P('Edge Object JSON:'),
                    html.Pre(
                        id='tap-edge-json-output',
                        style=styles['json-output']
                    )
                ])
            ])
        ]),

    ]),
    html.Div(className='eight columns', children=[
        cyto.Cytoscape(
            id='cytoscape',
            elements=default_elements,
            stylesheet=default_stylesheet,
            style={
                'height': '50vh',
                'width': '100%'
            }
        )
    ]),
])


# ############################## CALLBACKS ####################################
@app.callback(Output('tap-node-json-output', 'children'),
              [Input('cytoscape', 'tapNode')])
def display_tap_node(data):
    return json.dumps(data, indent=2)


@app.callback(Output('tap-edge-json-output', 'children'),
              [Input('cytoscape', 'tapEdge')])
def display_tap_edge(data):
    return json.dumps(data, indent=2)


@app.callback(Output('cytoscape', 'layout'),
              [Input('dropdown-layout', 'value')])
def update_cytoscape_layout(layout):
    return {'name': layout}


@app.callback(Output('cytoscape', 'elements'),
              [Input('cytoscape', 'tapNodeData')],
              [State('cytoscape', 'elements'),
               State('radio-expand', 'value')])
def generate_elements(nodeData, elements, expansion_mode):
    if not nodeData:
        return default_elements

    # If the node has already been expanded, we don't expand it again
    if nodeData.get('expanded'):
        return elements

    # This retrieves the currently selected element, and tag it as expanded
    for element in elements:
        if nodeData['id'] == element.get('data').get('id'):
            element['data']['expanded'] = True
            break

    if expansion_mode == 'followers':

        followers_nodes = followers_node_di.get(nodeData['id'])
        followers_edges = followers_edges_di.get(nodeData['id'])

        if followers_nodes:
            for node in followers_nodes:
                node['classes'] = 'followerNode'
            elements.extend(followers_nodes)

        if followers_edges:
            for follower_edge in followers_edges:
                follower_edge['classes'] = 'followerEdge'
            elements.extend(followers_edges)

    elif expansion_mode == 'following':

        following_nodes = following_node_di.get(nodeData['id'])
        following_edges = following_edges_di.get(nodeData['id'])

        if following_nodes:
            for node in following_nodes:
                if node['data']['id'] != genesis_node['data']['id']:
                    node['classes'] = 'followingNode'
                    elements.append(node)

        if following_edges:
            for follower_edge in following_edges:
                follower_edge['classes'] = 'followingEdge'
            elements.extend(following_edges)

    return elements


port = '8299'
host = '127.0.0.1'

app.run_server(mode='inline',port = port, host = host)

In [19]:
default_elements

[{'data': {'id': '108082478497335384404', 'label': 'User #84404'},
  'classes': 'genesis'}]